In [1]:
##FAKE NEWS CLASSIFIER

In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("/content/train.csv")

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
#droping the nan values
df=df.dropna()
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [8]:
df.shape

(18285, 5)

In [9]:
#independent and dependent variable
X=df.drop(['label'],axis=1)
y=df['label']

In [10]:
print(X.shape)
print(y.shape)

(18285, 4)
(18285,)


In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
### Vocabulary size
voc_size=5000

In [13]:
##Onehot Representation

In [14]:
messages=X.copy()
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
messages.reset_index(inplace=True)
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [22]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [23]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4321, 2406, 3376, 2404, 572, 4197, 3447, 163, 4866, 4215],
 [2245, 1778, 3873, 1435, 4175, 485, 1850],
 [2184, 193, 2045, 2674],
 [3780, 3948, 1615, 3768, 1623, 2808],
 [2362, 4175, 2715, 2748, 4222, 3502, 4175, 3749, 827, 2550],
 [1832,
  1474,
  3234,
  4362,
  1000,
  454,
  3087,
  2259,
  3222,
  2733,
  2748,
  3731,
  3043,
  3223,
  1850],
 [4740, 4686, 177, 636, 4858, 1013, 1676, 721, 2068, 1398, 1822],
 [100, 1123, 4954, 384, 4894, 1765, 454, 598, 2068, 1398, 1822],
 [4331, 4378, 215, 3162, 2924, 1500, 192, 2400, 454, 2860],
 [2668, 1371, 1045, 567, 4162, 279, 1807, 2055],
 [731, 2238, 46, 3880, 2459, 1536, 4928, 1154, 178, 1553, 1780],
 [3768, 971, 572, 1500, 454, 4894],
 [2862, 2901, 1529, 1558, 4469, 4945, 4941, 2765, 570],
 [3238, 2955, 1672, 4725, 760, 3741, 1152, 2068, 1398, 1822],
 [4341, 1748, 878, 1160, 1161, 2068, 1398, 1822],
 [4289, 460, 3764, 1457, 3500, 4155, 3710, 1101, 3898, 2040],
 [4760, 2556, 1778],
 [3104, 1759, 608, 2311, 454, 2736, 667, 1850],
 [91, 18

In [24]:
##Embedding Representation

In [25]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  163 4866 4215]
 [   0    0    0 ... 4175  485 1850]
 [   0    0    0 ...  193 2045 2674]
 ...
 [   0    0    0 ... 2068 1398 1822]
 [   0    0    0 ... 1558 2023 3271]
 [   0    0    0 ... 1229 1644 3367]]


In [26]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4321,
       2406, 3376, 2404,  572, 4197, 3447,  163, 4866, 4215], dtype=int32)

In [27]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
len(embedded_docs),y.shape

(18285, (18285,))

In [29]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [30]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [31]:
#MODEL TRAINING

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [33]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 34ms/step - loss: 0.3368 - accuracy: 0.8429 - val_loss: 0.2053 - val_accuracy: 0.9168
Epoch 2/10
192/192 [==============================] - 6s 31ms/step - loss: 0.1422 - accuracy: 0.9461 - val_loss: 0.1915 - val_accuracy: 0.9229
Epoch 3/10
192/192 [==============================] - 6s 31ms/step - loss: 0.1033 - accuracy: 0.9639 - val_loss: 0.2075 - val_accuracy: 0.9186
Epoch 4/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0782 - accuracy: 0.9718 - val_loss: 0.2279 - val_accuracy: 0.9130
Epoch 5/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0511 - accuracy: 0.9836 - val_loss: 0.2683 - val_accuracy: 0.9170
Epoch 6/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0345 - accuracy: 0.9900 - val_loss: 0.2950 - val_accuracy: 0.9147
Epoch 7/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0257 - accuracy: 0.9917 - val_loss: 0.4079 - val_accuracy: 0.9162

In [35]:
#Performance Matrix and Accuracy
y_pred = (model.predict(X_test) > 0.5).astype("int32")

In [36]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3131,  288],
       [ 234, 2382]])

In [37]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9135045567522784

In [38]:
#ADDING A DROPOUT RATIO
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Dropout(0.3))
model1.add(LSTM(100))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [43]:
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 35ms/step - loss: 0.3347 - accuracy: 0.8416 - val_loss: 0.1981 - val_accuracy: 0.9145
Epoch 2/10
192/192 [==============================] - 6s 33ms/step - loss: 0.1521 - accuracy: 0.9386 - val_loss: 0.2065 - val_accuracy: 0.9201
Epoch 3/10
192/192 [==============================] - 6s 33ms/step - loss: 0.1173 - accuracy: 0.9558 - val_loss: 0.2124 - val_accuracy: 0.9163
Epoch 4/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0909 - accuracy: 0.9655 - val_loss: 0.2718 - val_accuracy: 0.9162
Epoch 5/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0736 - accuracy: 0.9744 - val_loss: 0.2784 - val_accuracy: 0.9135
Epoch 6/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0605 - accuracy: 0.9808 - val_loss: 0.3227 - val_accuracy: 0.9162
Epoch 7/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0487 - accuracy: 0.9834 - val_loss: 0.3095 - val_accuracy: 0.9057

In [45]:
#Performance Matrix and Accuracy
y_pred = (model1.predict(X_test) > 0.5).astype("int32")

In [46]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3144,  275],
       [ 289, 2327]])

In [47]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9065451532725767